In [ ]:
import librosa
import numpy as np
from evoaudio.individual import BaseIndividual
from evoaudio.mutations import Mutator
from evoaudio.sample_library import SampleLibrary
from evoaudio.pitch import Pitch, DrumHit

sample_lib = SampleLibrary()

pitch_dict = {pitch.name: pitch.value for pitch in Pitch}
drum_dict = {DrumHit.name: DrumHit.value + 73 for DrumHit in DrumHit}

pitch_dict.update(drum_dict)

instruments = list(sample_lib.instruments.keys())

instrument_dict = {}
for i in range(len(instruments)):
    instrument_dict[instruments[i]] = i+1

style_list = []
for i in instruments:
    style_list = style_list + list(sample_lib.instruments[i].styles)

style_dict = {}
for i in range(len(style_list)):
    style_dict[style_list[i]] = i+1

def getLabel(x):
    label = []
    for n in range(5):
        if len(x.samples) > n:
            sampleTuple = [instrument_dict[str(x.samples[n].instrument)], style_dict[str(x.samples[n].style)], pitch_dict[str(x.samples[n].pitch.name)]]
            label = label + sampleTuple
        else:
            label = label + [0,0,0]
    return np.array(label)

def getSpec(x):
    mel_data = x.to_mixdown()[:(22050*3)]
    
    mel_spectrogram = librosa.feature.melspectrogram(y=mel_data, n_mels=128, hop_length=512)
    dbscale_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    current_length = dbscale_mel_spectrogram.shape[1]
    if current_length < 130:
            
        pad_width = 130 - current_length
        dbscale_mel_spectrogram = np.pad(dbscale_mel_spectrogram, ((0, 0), (0, pad_width)), mode='constant')

    return dbscale_mel_spectrogram


mutator = Mutator(sample_lib)
spectrograms = []
labels = []
max_time_steps = 0
for i in range(256):
    if i%1 == 0:
        print(i)
    
    x = BaseIndividual.create_random_individual(sample_lib=sample_lib) 
    spectrograms.append(getSpec(x))
    labels.append(getLabel(x))

    for j in range(0):
        mutation = np.random.choice([mutator.mutate_instrument, mutator.mutate_pitch, mutator.mutate_n_samples])
        x_copy = BaseIndividual.from_copy(x)
        x_mut = mutation(x_copy)
        spectrograms.append(getSpec(x_mut))
        labels.append(getLabel(x_mut))

spectrograms_array = np.array(spectrograms)
labels_array = np.array(labels)

np.save('spectrograms.npy', spectrograms_array)
np.save('labels.npy', labels_array)